In [1]:
import numpy as np
import os

In [8]:
def detect_namespace(namespace:str):
    content = namespace[namespace.find("(")+1:-1].split(", ")
    arg_dict = {}
    for arg in content:
        arg_type, arg_value = arg.split("=")
        arg_dict[arg_type] = arg_value
    return arg_dict

def get_acc(line:str, delimiter=":", precision=4):
    index = line.find(delimiter) + len(delimiter)
    acc = line[index:index + 3 + precision]
    return float(acc)

def get_std(line:str, delimiter="std:", precision=4):
    index = line.find(delimiter) + len(delimiter)
    acc = line[index:index + 3 + precision]
    return float(acc)

def get_rm(line:str, delimiter="removed", precision=4):
    index = line.find(delimiter) + len(delimiter)
    acc = line[index:index + 3 + precision]
    return float(acc)

def parse_file(f):
    FT_max = 0
    FT_std_max = 0
    RM_max = 0
    for line in f:
        if line.find("Namespace") != -1:
            arg_dict = detect_namespace(line)
        if line.find("No mask noise average acc") != -1:
            NM = get_acc(line)
            NM_std = get_std(line)
        if line.find("Finetune noise average acc") != -1:
            FT = get_acc(line)
            FT_std = get_std(line)
            if FT > FT_max:
                FT_max = FT
                FT_std_max = FT_std
                RM_max = RM
        if line.find("Total weights removed") != -1:
            RM = get_rm(line)
    return arg_dict, NM, FT_max, NM_std, FT_std_max, RM_max

def sort_keys(keys):
    keys = np.array(list(keys))
    sb_list = []
    hp_list = []
    tmp_sb = []
    for i in keys:
        tmp_sb.append(i[1])
    arg = np.argsort(tmp_sb)
    this_list = []
    that_list = []
    ttt = 0
    for index in range(len(arg)):
        i = keys[arg[index]][1]
        j = keys[arg[index]][0]
        if i != ttt:
            if this_list != []:
                sb_list.append(this_list)
                hp_list.append(that_list)
            ttt = i
            this_list = []
            that_list = []
        this_list.append(i)
        that_list.append(j)
    if this_list != []:
        sb_list.append(this_list)
        hp_list.append(that_list)
    for i in range(len(hp_list)):
        hp_list[i] = np.sort(hp_list[i]).tolist()
    return sb_list, hp_list

In [9]:
class PResultItem():
    def __init__(self, main_key:float):
        self.main_key = main_key
        self.NM_list = []
        self.FT_list = []
        self.NM_std_list = []
        self.FT_std_list = []
        self.RM_list = []
    
    def add_result(self, NM:float, FT:float, NM_std:float, FT_std:float, RM:float):
        assert len(self.NM_list) == len(self.FT_list)
        self.NM_list.append(NM)
        self.FT_list.append(FT)
        self.NM_std_list.append(NM_std)
        self.FT_std_list.append(FT_std)
        self.RM_list.append(RM)

class PResult():
    def __init__(self):
        self.dict = {}
    
    def add_result(self, arg_dict:dict, NM:float, FT:float, NM_std:float, FT_std:float, RM:float, main_key_str="mask_p", minor_key_str="alpha"):
        # print(arg_dict)
        main_key = float(arg_dict[main_key_str])
        minor_key = float(arg_dict[minor_key_str])
        key = (main_key, minor_key)
        if not(key in self.dict.keys()):
            self.dict[key] = PResultItem(key)
        self.dict[key].add_result(NM, FT, NM_std, FT_std, RM)

In [13]:
dir_path = "./results/noise/CIFARQ1000"
file_blob = os.listdir(dir_path)
main_key_str = "mask_p"
minor_key_str = "alpha"
# main_key_str = "train_epoch"
results = PResult()
for fn in file_blob:
    # cfn = os.path.join(dir_path, fn)
    # with open(cfn) as f:
    #     f = f.read().split("\n")
    # arg_dict, NM, FT, NM_std, FT_std, RM = parse_file(f)
    try:
        cfn = os.path.join(dir_path, fn)
        with open(cfn) as f:
            f = f.read().split("\n")
        try:
            arg_dict, NM, FT, NM_std, FT_std, RM = parse_file(f)
        except:
            print(fn)
        results.add_result(arg_dict, NM, FT, NM_std, FT_std, RM, main_key_str, minor_key_str)
    except:
        print(fn)

adv_train.o856446.6
adv_train.o856446.8
adv_train.o856446.7
adv_train.o856446.5


In [14]:
sb, hp = sort_keys(results.dict.keys())
minor_key = 0
for ii in range(len(sb)):
    print(f"{minor_key_str} = {sb[ii][0]}")
    for jj in range(len(sb[ii])):
        key_item = (hp[ii][jj], sb[ii][jj])
        i = results.dict[key_item]
        NM = np.mean(i.NM_list)
        FT = np.mean(i.FT_list)
        NM_std = np.mean(i.NM_std_list)
        FT_std = np.mean(i.FT_std_list)
        RM = np.mean(i.RM_list)
        gain= FT - NM
        red = NM_std - FT_std
        print(f"{main_key_str} = {hp[ii][jj]:.0e}, ACC NM: {NM:.4f}, FT: {FT:.4f}, Gain: {gain:.4f},   STD NM: {NM_std:.4f}, FT: {FT_std:.4f}, Red: {red:.4f}")
        # print(RM)

alpha = 4.0
mask_p = 1e-05, ACC NM: 0.8288, FT: 0.8340, Gain: 0.0052,   STD NM: 0.0052, FT: 0.0047, Red: 0.0005
mask_p = 1e-04, ACC NM: 0.8288, FT: 0.8341, Gain: 0.0053,   STD NM: 0.0052, FT: 0.0047, Red: 0.0005
mask_p = 1e-03, ACC NM: 0.8288, FT: 0.8346, Gain: 0.0058,   STD NM: 0.0052, FT: 0.0046, Red: 0.0006
mask_p = 1e-02, ACC NM: 0.8288, FT: 0.8329, Gain: 0.0041,   STD NM: 0.0052, FT: 0.0045, Red: 0.0007
alpha = 6.0
mask_p = 1e-05, ACC NM: 0.8288, FT: 0.8340, Gain: 0.0052,   STD NM: 0.0052, FT: 0.0047, Red: 0.0005
mask_p = 1e-04, ACC NM: 0.8287, FT: 0.8341, Gain: 0.0054,   STD NM: 0.0052, FT: 0.0047, Red: 0.0005
mask_p = 1e-03, ACC NM: 0.8288, FT: 0.8348, Gain: 0.0061,   STD NM: 0.0052, FT: 0.0046, Red: 0.0006
mask_p = 7e-03, ACC NM: 0.8288, FT: 0.8374, Gain: 0.0086,   STD NM: 0.0052, FT: 0.0044, Red: 0.0008
mask_p = 9e-03, ACC NM: 0.8288, FT: 0.8370, Gain: 0.0083,   STD NM: 0.0052, FT: 0.0043, Red: 0.0009
mask_p = 1e-02, ACC NM: 0.8286, FT: 0.8369, Gain: 0.0082,   STD NM: 0.0052, 